In [1]:
import os
import torch
from functools import reduce
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score, mean_absolute_error, mean_squared_log_error
import numpy as np
import pandas as pd

In [2]:
seed = 42

torch.manual_seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [3]:
df = pd.read_csv("../Data/Fixed-Fixed.csv")
df.head(18)

,depth,l1,l2,Mode,Frequency,Max_Deflection,Average_Deflection
0,0.0,0,0,1,52.976,18.084,9.0351
1,0.0,0,0,2,145.920,17.293,9.3771
2,0.0,0,0,3,285.990,17.430,9.5803
3,0.0,0,0,4,323.400,28.366,10.8720
4,0.0,0,0,5,472.810,17.537,9.7190
5,0.0,0,0,6,489.850,17.734,9.2553
6,0.5,60,110,1,52.618,18.051,7.3559
7,0.5,60,110,2,144.890,17.276,8.1888
8,0.5,60,110,3,283.940,17.488,9.0011
9,0.5,60,110,4,322.780,28.340,9.2718


# Preprocessing

In [4]:
X = df[["depth", "l1", "l2", "Mode"]].values
Y = df[["Frequency", "Max_Deflection", "Average_Deflection"]].values

In [5]:
# Split the data

_X_train, _X_test, _y_train, _y_test = train_test_split(X, Y, test_size=0.30, random_state=seed)

In [6]:
# Standard Scalar

std_X = StandardScaler()
std_X.fit(_X_train[:, :-1]) # Fitting on the Numerical Part of the Data

std_Y = StandardScaler()
std_Y.fit(_y_train) # Fitting on the Numerical Part of the Data

StandardScaler(copy=True, with_mean=True, with_std=True)

In [7]:
get_feature = lambda x: np.hstack((std_X.transform(x[:, :-1]), np.eye(6)[x[:, -1].astype(np.int8) - 1]))

In [8]:
get_train_output = lambda y: std_Y.transform(y)
get_output = lambda y: Variable(torch.Tensor(std_Y.inverse_transform(y.detach().numpy())))

In [9]:
explained_variance_score_tensor = lambda y_, y: explained_variance_score(y_.detach().numpy(), y.detach().numpy())
mean_absolute_error_tensor = lambda y_, y: mean_absolute_error(y_.detach().numpy(), y.detach().numpy())
mean_squared_log_error_tensor = lambda y_, y: mean_squared_log_error(y_.detach().numpy(), y.detach().numpy())

# Data

In [10]:
X_train = Variable(torch.Tensor(get_feature(_X_train)))
y_train = Variable(torch.Tensor(get_train_output(_y_train)))

In [11]:
X_test = Variable(torch.Tensor(get_feature(_X_test)))
y_test = Variable(torch.Tensor(_y_test))

# Neural Networks

### 1. Network Design

In [12]:
class NeuralNet(torch.nn.Module):
    """
    Custome Neural Network
    """
    def __init__(self, n_feature, n_hidden1, n_output, seed=seed):
        super(NeuralNet, self).__init__()
        # Hidden Layer
        self.hidden1 = torch.nn.Linear(n_feature, n_hidden1)
        
        # Output Layer
        self.predict = torch.nn.Linear(n_hidden1, n_output)
        
        # Initializae all layers with Xavir Initialziation
        torch.manual_seed(seed)
        torch.nn.init.xavier_normal_(self.hidden1.weight)
        torch.manual_seed(seed)
        torch.nn.init.xavier_normal_(self.predict.weight)

    def forward(self, x):
        # Activation of Outputs
        x = F.relu(self.hidden1(x))
        
        # Linear Otput
        x = self.predict(x)
        return x

### 2. Training

In [13]:
sum_py = lambda x,y: x+y

In [14]:
# L2 Regularization

l2_reg = lambda net: reduce(sum_py, [torch.sum(x**2) for x in net.parameters()])

In [15]:
def fit(net, x, y, epochs=20, alpha=0.1, lmbda=0.02, seed=seed):
    """
    Function to Train Neural Netowrk
    """
    torch.manual_seed(seed)
    optimizer = torch.optim.SGD(net.parameters(), lr=alpha)
    loss_func = torch.nn.MSELoss()
    
    for epoch in range(epochs):
        y_pred = net.forward(x)
        loss = loss_func(y_pred, y) + lmbda*l2_reg(net)

        optimizer.zero_grad()   # Clear Gradients For Next Epoch
        loss.backward()         # Backpropagation, Compute Gradients
        optimizer.step()        # Apply Gradients
        
    return net

### 3. Evaluation [Training with All Data]

In [16]:
net_f = NeuralNet(n_feature = X_train.shape[-1], 
                n_hidden1 = 10,
                n_output = y_train.shape[-1]) 

net_f = fit(net_f, X_train, y_train, epochs=500, alpha=1.0, lmbda=0.01)

y_pred = net_f.forward(X_test)
y_pred = get_output(y_pred)

In [17]:
explained_variance_score_tensor(y_pred, y_test)

0.7747865319252014

In [18]:
mean_absolute_error_tensor(y_pred, y_test)

7.005413

In [19]:
mean_squared_log_error_tensor(y_pred, y_test)

0.022167342

### 4. Batch Training

In [20]:
def fit_in_batches(net, x, y, epochs=20, batch_size = 32, alpha=0.1, lmbda=0.02, seed=seed):
    for epoch in range(epochs):
        for idx in range(0, len(x)+1, batch_size):
            net = fit(net, 
                      x[idx:min(idx+32, len(x))], 
                      y[idx:min(idx+32, len(x))], 
                      1, 
                      alpha, 
                      lmbda, 
                      seed)
    return net

### 5. Evaluation [Data in Batches]

In [21]:
net_b = NeuralNet(n_feature = X_train.shape[-1], 
                n_hidden1 = 10,
                n_output = y_train.shape[-1])

net_b = fit_in_batches(net_b, X_train, y_train, batch_size=128, epochs=300, alpha=1.0, lmbda=0.01)

y_pred = net_b.forward(X_test)
y_pred = get_output(y_pred)

In [22]:
explained_variance_score_tensor(y_pred, y_test)

0.7593264579772949

In [23]:
mean_absolute_error_tensor(y_pred, y_test)

6.8614125

In [24]:
mean_squared_log_error_tensor(y_pred, y_test)

0.004843051

# Final Testing Phase

In this phase separeate simulations were done with new set of points generated random;y with different random seed.

In [25]:
df_test = pd.read_csv("../Data/Fixed-Fixed-Test.csv")
df_test.head(18)

,depth,l1,l2,Mode,Frequency,Max_Deflection,Average_Deflection
0,0.5,63,510,1,52.613,7.5540,3.8030
1,0.5,63,510,2,144.790,7.2355,3.6191
2,0.5,63,510,3,283.540,7.3149,4.0421
3,0.5,63,510,4,322.530,11.8550,4.5453
4,0.5,63,510,5,468.260,7.4041,3.8655
5,0.5,63,510,6,489.250,7.4189,3.8899
6,0.5,255,742,1,52.730,7.5491,3.8768
7,0.5,255,742,2,145.000,7.2373,4.5900
8,0.5,255,742,3,284.560,7.3335,4.6257
9,0.5,255,742,4,322.740,11.8650,4.7365


In [26]:
test_X = df_test[["depth", "l1", "l2", "Mode"]].values
test_X =  Variable(torch.Tensor(get_feature(test_X)))

test_Y = df_test[["Frequency", "Max_Deflection", "Average_Deflection"]].values
test_Y = Variable(torch.Tensor(test_Y))

### Test Result for NN fitted full data.

In [27]:
test_pred_y = net_f.forward(test_X)
test_pred_y = get_output(test_pred_y)

In [28]:
explained_variance_score_tensor(test_pred_y, test_Y)

0.7592843174934387

In [29]:
mean_absolute_error_tensor(test_pred_y, test_Y)

12.451977

In [30]:
mean_squared_log_error_tensor(test_pred_y, test_Y)

0.43317863

### Test Result for NN fitted with date in batches

In [31]:
test_pred_y = net_b.forward(test_X)
test_pred_y = get_output(test_pred_y)

In [32]:
explained_variance_score_tensor(test_pred_y, test_Y)

0.7640375097592672

In [33]:
mean_absolute_error_tensor(test_pred_y, test_Y)

12.48317

In [34]:
mean_squared_log_error_tensor(test_pred_y, test_Y)

0.3985153